In [2]:
%pip install pandas

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd

In [4]:
def read_sqlite(dbfile):
    import sqlite3
    from pandas import read_sql_query, read_sql_table

    with sqlite3.connect(dbfile) as dbcon:
        tables = list(read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", dbcon)['name'])
        out = {tbl : read_sql_query(f"SELECT * from {tbl}", dbcon) for tbl in tables}

    return out

In [5]:
db = read_sqlite("../input_files/steel_moment_axial_combos_oct6_2.db")

In [6]:
db.keys()

dict_keys(['Wshapes_Table6_1', 'wShapes'])

In [7]:
w_shapes = db['wShapes']
w_shapes.shape

(2589600, 9)

In [8]:
Wshapes_Table6_1 = db['Wshapes_Table6_1']
Wshapes_Table6_1.shape

(6139000, 9)

In [9]:
w_shapes.head()

,profile,length_ft,weight_plf,Cb,Pr_over_Pc,Pr_k,Pc_k,Mmax_ftk,Mc_ftk
0,W44X290,0,290.63328,1.0,0.0,0.0,2397.118549,3517.964072,3517.964072
1,W44X290,0,290.63328,1.1,0.0,0.0,2397.118549,3517.964072,3517.964072
2,W44X290,0,290.63328,1.2,0.0,0.0,2397.118549,3517.964072,3517.964072
3,W44X290,0,290.63328,1.3,0.0,0.0,2397.118549,3517.964072,3517.964072
4,W44X290,0,290.63328,1.4,0.0,0.0,2397.118549,3517.964072,3517.964072


In [10]:
Wshapes_Table6_1.head()

,profile,length_ft,weight_plf,Cb,Pr_over_Pc,Pr_k,Pc_k,Mmax_ftk,Mc_ftk
0,W44X335,0,335,1.0,0.0,0.0,2890.17,4040.4,4040.4
1,W44X335,0,335,1.1,0.0,0.0,2890.17,4040.4,4040.4
2,W44X335,0,335,1.2,0.0,0.0,2890.17,4040.4,4040.4
3,W44X335,0,335,1.3,0.0,0.0,2890.17,4040.4,4040.4
4,W44X335,0,335,1.4,0.0,0.0,2890.17,4040.4,4040.4


In [11]:
primary_keys = ['profile', 'length_ft', 'Cb', 'Pr_over_Pc']

merged_df = w_shapes.merge(Wshapes_Table6_1, on=primary_keys, suffixes=('_w_shapes', '_Wshapes_Table6_1'))

In [12]:
merged_df.shape

(514680, 14)

In [13]:
merged_df.head()

,profile,length_ft,weight_plf_w_shapes,Cb,Pr_over_Pc,Pr_k_w_shapes,Pc_k_w_shapes,Mmax_ftk_w_shapes,Mc_ftk_w_shapes,weight_plf_Wshapes_Table6_1,Pr_k_Wshapes_Table6_1,Pc_k_Wshapes_Table6_1,Mmax_ftk_Wshapes_Table6_1,Mc_ftk_Wshapes_Table6_1
0,W44X290,0,290.63328,1.0,0.0,0.0,2397.118549,3517.964072,3517.964072,290,0.0,2398.08,3513.39,3513.39
1,W44X290,0,290.63328,1.1,0.0,0.0,2397.118549,3517.964072,3517.964072,290,0.0,2398.08,3513.39,3513.39
2,W44X290,0,290.63328,1.2,0.0,0.0,2397.118549,3517.964072,3517.964072,290,0.0,2398.08,3513.39,3513.39
3,W44X290,0,290.63328,1.3,0.0,0.0,2397.118549,3517.964072,3517.964072,290,0.0,2398.08,3513.39,3513.39
4,W44X290,0,290.63328,1.4,0.0,0.0,2397.118549,3517.964072,3517.964072,290,0.0,2398.08,3513.39,3513.39


In [14]:
to_compare_columns = ['Pr_k', 'Pc_k', 'Mmax_ftk', 'Mc_ftk']

passes = 0
failures = 0

failed_items = []

for index, row in merged_df.iterrows():

    # check the difference for each column in to_compare_columns
    differences = [abs(row[f'{col}_w_shapes'] - row[f'{col}_Wshapes_Table6_1']) for col in to_compare_columns]

    if all(diff <= 0.05 * max(row[f'{col}_w_shapes'], row[f'{col}_Wshapes_Table6_1']) for diff, col in zip(differences, to_compare_columns)):
        passes += 1
    else:
        failures += 1
        failed_items.append(index)

print('Threshold constant: 5')
print('Number of passes:', passes)
print('Number of failures:', failures)
print("Failed Items:")
merged_df.loc[failed_items]

Threshold constant: 5
Number of passes: 514680
Number of failures: 0
Failed Items:


,profile,length_ft,weight_plf_w_shapes,Cb,Pr_over_Pc,Pr_k_w_shapes,Pc_k_w_shapes,Mmax_ftk_w_shapes,Mc_ftk_w_shapes,weight_plf_Wshapes_Table6_1,Pr_k_Wshapes_Table6_1,Pc_k_Wshapes_Table6_1,Mmax_ftk_Wshapes_Table6_1,Mc_ftk_Wshapes_Table6_1


In [15]:
def calculate_percent_difference(x1, x2):
    if x1 + x2 == 0:
        return 0
    return (abs(x1 - x2) / ((x1 + x2) / 2)) * 100

In [16]:
to_compare_columns = ['Pr_k', 'Pc_k', 'Mmax_ftk', 'Mc_ftk']

passes2 = 0
failures2 = 0
threshold = 5.0
failed_items2 = []

for index, row in merged_df.iterrows():

    differences = [calculate_percent_difference(row[f'{col}_w_shapes'], row[f'{col}_Wshapes_Table6_1']) for col in to_compare_columns]

    if all(diff <= threshold for diff in differences):
        passes2 += 1
    else:
        failures2 += 1
        failed_items2.append(index)

print('Threshold: ', threshold)
print('Number of passes 2:', passes2)
print('Number of failures 2:', failures2)
print("Failed Items 2:")
merged_df.loc[failed_items2]

Threshold:  5.0
Number of passes 2: 514680
Number of failures 2: 0
Failed Items 2:


,profile,length_ft,weight_plf_w_shapes,Cb,Pr_over_Pc,Pr_k_w_shapes,Pc_k_w_shapes,Mmax_ftk_w_shapes,Mc_ftk_w_shapes,weight_plf_Wshapes_Table6_1,Pr_k_Wshapes_Table6_1,Pc_k_Wshapes_Table6_1,Mmax_ftk_Wshapes_Table6_1,Mc_ftk_Wshapes_Table6_1


In [17]:
to_compare_columns = ['Pr_k', 'Pc_k', 'Mmax_ftk', 'Mc_ftk']
passes3 = 0
failures3 = 0
threshold = 5.0
failed_items_details3 = []

for index, row in merged_df[:500].iterrows():
    differences = [calculate_percent_difference(row[f'{col}_w_shapes'], row[f'{col}_Wshapes_Table6_1']) for col in to_compare_columns]

    # Check if all differences are within 5%
    if all(diff <= threshold for diff in differences):
        passes3 += 1
    else:
        failures3 += 1
        print(to_compare_columns)
        print(differences)
        print(row)
        failed_items_details3.append({
            'index': index,
            'differences': {col: diff for col, diff in zip(to_compare_columns, differences)}
        })

# Print the number of passes and failures
print('Number of passes 3 for the first 500 rows of merged_df:', passes3)
print('Number of failures 3 for the first 500 rows of merged_df:', failures3)

# Print details of failed items
print("\nFailed Items Details for the first 500 rows of merged_df:")
for failed_item in failed_items_details3:
    print(f"Failed item at index {failed_item['index']}:")
    for col, diff in failed_item['differences'].items():
        if diff > threshold:
            print(f"  {col}: {diff:.2f}%")


Number of passes 3 for the first 500 rows of merged_df: 500
Number of failures 3 for the first 500 rows of merged_df: 0

Failed Items Details for the first 500 rows of merged_df:


In [18]:
numeric_columns = ['Pr_k', 'Pc_k', 'Mmax_ftk', 'Mc_ftk']
pass_counts = {col: 0 for col in numeric_columns}
fail_counts = {col: 0 for col in numeric_columns}

# Create a list to store the details of failed items
failed_items_details_df = []

# Iterate through each row
for index, row in merged_df.iterrows():
    # Check the difference for each numeric column
    differences = [calculate_percent_difference(row[f'{col}_w_shapes'], row[f'{col}_Wshapes_Table6_1']) for col in numeric_columns]

    # Check if differences for each column are within 5%
    for col, diff in zip(numeric_columns, differences):
        if diff <= 5.0:
            pass_counts[col] += 1
        else:
            fail_counts[col] += 1

# Print the number of passes and failures for each column
print('Pass/Fail counts for each column:')
for col in numeric_columns:
    print(f'{col}: Passes={pass_counts[col]}, Failures={fail_counts[col]}')

Pass/Fail counts for each column:
Pr_k: Passes=514680, Failures=0
Pc_k: Passes=514680, Failures=0
Mmax_ftk: Passes=514680, Failures=0
Mc_ftk: Passes=514680, Failures=0
